<a href="https://colab.research.google.com/github/HitsujiAura/AbstractArtGAN/blob/master/stylegan2-ada-pytorch-landscape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Custom Training StyleGan2-ADA

StyleGAN2-ADA only work with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [1]:
!nvidia-smi

Sat Jun  5 07:44:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Repo to Google Drive

Colab is a little funky with training. I’ve found the best way to do this is to install the repo directly into your Google Drive folder.

First, mount your Drive to the Colab notebook: 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.

In [4]:
import os
if os.path.isdir("/content/drive/My Drive/stylegan2-ada"):
    %cd "/content/drive/My Drive/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/My Drive/"
    !git clone https://github.com/robgon-art/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/My Drive/stylegan2-ada


In [ ]:
# %cd "/content/drive/My Drive/colab-sg2-ada/stylegan2-ada"
# !git config --global user.name "test"
# !git config --global user.email "test@test.com"
# !git fetch origin
# !git checkout origin/main -- train.py

## Convert dataset to .tfrecords

**Note: You only need to do this once per dataset. If you have already run this and are returning to conntinue training, skip these cells.**

Next we need to convert our image dataset to a format that StyleGAN2-ADA can read from. There are two options here. You can upload your dataset directly to Colab (as a zipped file), or you can upload it to Drive directly and read it from there.

In [4]:
#if you manually uploaded your dataset to Colab, unzip it
zip_path = "/content/drive/MyDrive/Datasets/impressionist_art.zip"

!unzip {zip_path} -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_a-copse-evening-1918.jpg  
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_early-spring-quebec-1923.jpg  
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_first-snow-algoma-country-1920.jpg  
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_frozen-lake-early-spring-algonquin-park-1914.jpg  
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_maple-woods-algoma-1920.jpg  
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_march-storm-georgian-bay-1920.jpg  
  inflating: /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/a-y-jackson_night-pine-island-1921.jpg  
  inflating: /content/content/drive/MyDrive/impres

Now that your image dataset is uploaded, we need to convert it to the `.tfrecords` format.

Depending on the resolution of your images and how many you have, this can take a while.

In [9]:
#update this to the path to your image folder
dataset_path = "/content/abstract_art_512"
#give your dataset a name
dataset_name = "abstract_art_512"

#you don't need to edit anything here
!python dataset_tool.py create_from_images  ./datasets/impressionist_art/ /content/content/drive/MyDrive/impressionist_landscapes_resized_1024/

Loading images from "/content/content/drive/MyDrive/impressionist_landscapes_resized_1024/"
Creating dataset "./datasets/impressionist_art/"
dataset_tool.py:96: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 5000 images.


## Train a custom model

We’re ready to start training! There are numerous arguments to training, what’s listed below are the most popular options. To see all the options, run the following cell.

In [5]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

     |████████████████████████████████| 204kB 13.8MB/s 
     |████████████████████████████████| 112kB 24.4MB/s 
     |████████████████████████████████| 26.9MB 1.3MB/s 


In [ ]:
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in
  the paper "Training Generative Adversarial
  Networks with Limited Data".

  Examples:

  # Train with custom dataset using 1 GPU.
  python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1

  # Train class-conditional CIFAR-10 using 2 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/cifar10.zip \
      --gpus=2 --cfg=cifar --cond=1

  # Transfer learn MetFaces from FFHQ using 4 GPUs.
  python train.py --outdir=~/training-runs --data=~/datasets/metfaces.zip \
      --gpus=4 --cfg=paper1024 --mirror=1 --resume=ffhq1024 --snap=10

  # Reproduce original StyleGAN2 config F.
  python train.py --outdir=~/training-runs --data=~/datasets/ffhq.zip \
      --gpus=8 --cfg=stylegan2 --mirror=1 --aug=noaug

  Base configs (--cfg):
    auto       Automatically select reasonable defaults based on resolution
               and GPU count. Good starting point for new datasets.
    stylegan2  R

In [ ]:
# !python train.py --outdir=./results/impressionist_art --data=./datasets/impressionist_art_1024.zip --gpus=1 --snap=10 --metrics=none --resume=./results/impressionist_art/00000-impressionist_art_1024-auto1/network-snapshot-000160.pkl
!python train.py --aug=ada --p 0.186 --mirror=1 --metrics=none --snap=2 --gpus=1 --data=./datasets/impressionist_art --outdir=./results --resume=./results/00000-impressionist_art-mirror-auto1-ada-p0.186/network-snapshot-000040.pkl

tcmalloc: large alloc 4294967296 bytes == 0x55873a40a000 @  0x7f2ea80ed001 0x7f2ea533054f 0x7f2ea5380b58 0x7f2ea5384b17 0x7f2ea5423203 0x558732f66d54 0x558732f66a50 0x558732fdb105 0x558732fd57ad 0x558732f68c9f 0x558732fa9d79 0x558732fa6cc4 0x558732f67559 0x558732fdb4f8 0x558732fd54ae 0x558732ea7e2c 0x558732fd7bb5 0x558732fd54ae 0x558732ea7e2c 0x558732fd7bb5 0x558732fd57ad 0x558732ea7e2c 0x558732fd7bb5 0x558732f6830a 0x558732fd63b5 0x558732fd54ae 0x558732fd51b3 0x55873309f182 0x55873309f4fd 0x55873309f3a6 0x558733076723
tcmalloc: large alloc 4294967296 bytes == 0x55883a40a000 @  0x7f2ea80eb1e7 0x7f2ea533046e 0x7f2ea5380c7b 0x7f2ea538135f 0x7f2ea5423103 0x558732f66d54 0x558732f66a50 0x558732fdb105 0x558732fd54ae 0x558732f683ea 0x558732fd732a 0x558732fd54ae 0x558732f683ea 0x558732fd732a 0x558732fd54ae 0x558732f683ea 0x558732fd732a 0x558732f6830a 0x558732fd63b5 0x558732fd54ae 0x558732f683ea 0x558732fda7f0 0x558732fd54ae 0x558732f683ea 0x558732fd732a 0x558732fd57ad 0x558732f68c9f 0x558732fa

### While it’s training...
**Once the above cell is running you should be training!**

Don’t close this tab! Colab needs to be open and running in order to continue training. Every ~15min or so a new line should get added to your output, indicated its still training. Depending on you `snapshot_count` setting you should see the results folder in your Google drive folder fill with both samples (`fakesXXXXXx.jpg`) and model weights (`network-snapshot-XXXXXX.pkl`). The samples are worth looking at while it trains but don’t get too worried about each individual sample.

If you chose a metric, you will also see scores for each snapshot. Don’t obsess over these! they are a guide, it can go up or down slightly for each snapshot. What you want to see is a gradual lowering of the score over time.

Once Colab shuts off, you can Reconnect the notebook and re-run every cell from top to bottom. Make sure you update the `resume_from` path to continue training from the latest model.

In [ ]:
!python generate.py --outdir=./results/art/ --trunc=1 --seeds=0-5 \
    --network=./results/impressionist_art/00000-impressionist_art_1024-auto1/network-snapshot-000160.pkl

Loading networks from "./results/impressionist_art/00000-impressionist_art_1024-auto1/network-snapshot-000160.pkl"...
Generating image for seed 0 (0/6) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/6) ...
Generating image for seed 2 (2/6) ...
Generating image for seed 3 (3/6) ...
Generating image for seed 4 (4/6) ...
Generating image for seed 5 (5/6) ...


In [ ]:
!python generate.py --outdir=./results/art/5 --trunc=1.4 --seeds=0-100 \
    --network=./results/00005-abstract_art_512-auto1-resumecustom/network-snapshot-000960.pkl

Loading networks from "./results/00005-abstract_art_512-auto1-resumecustom/network-snapshot-000960.pkl"...
Generating image for seed 0 (0/101) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Generating image for seed 1 (1/101) ...
Generating image for seed 2 (2/101) ...
Generating image for seed 3 (3/101) ...
Generating image for seed 4 (4/101) ...
Generating image for seed 5 (5/101) ...
Generating image for seed 6 (6/101) ...
Generating image for seed 7 (7/101) ...
Generating image for seed 8 (8/101) ...
Generating image for seed 9 (9/101) ...
Generating image for seed 10 (10/101) ...
Generating image for seed 11 (11/101) ...
Generating image for seed 12 (12/101) ...
Generating image for seed 13 (13/101) ...
Generating image for seed 14 (14/101) ...
Generating image for seed 15 (15/101) ...
Generating image for seed 16 (16/101) ...
Generating image for seed 17 (17/101) ...
Generating image for seed 18 (18/101) ...
Generati